<a href="https://colab.research.google.com/github/eileenblum/Lyric-Generator/blob/main/Lyric_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#References: 
# guru99 website is a major reference for this code
# also  https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
# and https://medium.com/swlh/sentiment-classification-using-word-embeddings-word2vec-aedf28fbb8ca
import pandas as pd
import numpy as np
import os
import string
import nltk
import gensim
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import word2vec

### New:
import tensorflow as tf

from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras import layers
import time 

from keras import backend
from keras.optimizers import RMSprop
from keras.initializers import RandomNormal
from keras.layers import BatchNormalization

!pip install tensorflow_addons

import tensorflow_addons as tfa
from IPython import display
###

from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import abc
from nltk.corpus import stopwords
from textblob import Word
from gensim.models import Word2Vec
sns.set_style("whitegrid")


##########################  1  #################################################
############# Loading the data #################################################
################################################################################
#establish working directory:
#os.chdir(r'C:\Users\hprob\Desktop\ErdosProjectMay2020\Sample_project\Music-Project-master\Music-Project-master\Lyrics\metal_lyrics')
#os.getcwd()
#entries = os.scandir(r'metal_lyrics')
#entries

###############  Gather the Metal Files  ################################
#Make sure to change this path to where YOU are storing lyrics in your computer    
Metal_name_and_text = {}
# Walking a directory tree and printing the names of the directories and files
for dirpath, dirnames, files in os.walk(r'C:\Users\hprob\Desktop\ErdosProjectMay2020\Sample_project\Music-Project-master\Music-Project-master\Lyrics\metal_lyrics'):
    print(f'Found directory: {dirpath}')
    for file_name in files:
        os.listdir()
        print("song:", file_name)
        with open(dirpath + r'\\' + file_name, "r", errors='ignore') as target_file:
            Metal_name_and_text[file_name] = target_file.read()
    Metal_data = (pd.DataFrame.from_dict(Metal_name_and_text, orient='index')
            .reset_index().rename(index=str, columns={'index': 'song_name', 0: 'lyrics'}))

Type = np.zeros(shape=(len(Metal_name_and_text),1 ))
Metal_data['type'] = Type


###############  Gather the Reggae Files  ###############################
Reggae_name_and_text = {}
DFReggae_name_and_text = pd.DataFrame([['delete','delete','delete','delete','delete']],
                                      columns= ['ARTIST_NAME', 'ARTIST_URL', 'SONG_NAME', 'SONG_URL', 'LYRICS'])

# Walking a directory tree and printing the names of the directories and files
for dirpath, dirnames, files in os.walk(r'C:\Users\hprob\Desktop\ErdosProjectMay2020\Sample_project\Music-Project-master\Music-Project-master\Lyrics\azlyrics-scraper'):
    for file_name in files:
        Reggae_name_and_text[file_name] = pd.read_csv(r'C:\Users\hprob\Desktop\ErdosProjectMay2020\Sample_project\Music-Project-master\Music-Project-master\Lyrics\azlyrics-scraper' + r'\\' + file_name, error_bad_lines=False) 
        temp = pd.read_csv(r'C:\Users\hprob\Desktop\ErdosProjectMay2020\Sample_project\Music-Project-master\Music-Project-master\Lyrics\azlyrics-scraper' + r'\\' + file_name, error_bad_lines=False)
        DFReggae_name_and_text = DFReggae_name_and_text.append(temp)  

del DFReggae_name_and_text['ARTIST_NAME']
del DFReggae_name_and_text['ARTIST_URL']
del DFReggae_name_and_text['SONG_URL']
DFReggae_name_and_text = DFReggae_name_and_text.reset_index()
DFReggae_name_and_text = DFReggae_name_and_text.drop( index = 0)
del DFReggae_name_and_text['index']

len(DFReggae_name_and_text)
Type = np.ones(shape=(len(DFReggae_name_and_text), 1 ))

Reggae_data = DFReggae_name_and_text.copy()

Reggae_data['type'] = Type####################################
Reggae_data.columns = ['song_name', 'lyrics', 'type']
del Type


##########################  2  #################################################
############# Preprocessing the Data ###########################################
################################################################################

##############  Unionizing data ####
#input pds Metal_data and Reggae_data
subMetal_data = Metal_data.sample(len(DFReggae_name_and_text))

Lyrics = subMetal_data.copy()
Lyrics = Lyrics.append(Reggae_data)  #<------------this is the data we will work with
Lyrics = Lyrics.dropna()
##########################################

############  Spliting into train and test ############
Lyrics_train, Lyrics_test, type_train, type_test = train_test_split(Lyrics[['song_name','lyrics']],
                                                                    Lyrics[['type']], 
                                                                    stratify=Lyrics[['type']], test_size=0.1)
#######################################################

################# Cleaning ##############################
stop = stopwords.words('english') #importing ENglish stop words
#cleaning and lemmatizing Lyrics_train:
Pro_Lyrics_train = pd.DataFrame(Lyrics_train).copy()
Pro_Lyrics_train['lyrics'] = Pro_Lyrics_train['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))#All the rows of the text in the data frame is checked for string punctuations, and these are filtered
Pro_Lyrics_train['lyrics'] = Pro_Lyrics_train['lyrics'].str.replace('[^\w\s]','')#REmoves dots using regular expressions
Pro_Lyrics_train['lyrics'] = Pro_Lyrics_train['lyrics'].apply(lambda x:' '.join(x.lower() for x in x.split())) #Converting text to lower case
Pro_Lyrics_train['lyrics'] = Pro_Lyrics_train['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))#Digits are removed from the text
Pro_Lyrics_train['lyrics'] = Pro_Lyrics_train['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if not x in stop))#Stop words are removed at this stage
Pro_Lyrics_train['lyrics'] = Pro_Lyrics_train['lyrics'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))#Words are filtered now, and different form of the same word is removed using lemmatization
Pro_Lyrics_list = []
for i in Pro_Lyrics_train['lyrics']:
     li = list(i.split(" "))
     Pro_Lyrics_list.append(li)	
     
Pro_Lyrics_train['words'] = Pro_Lyrics_list #Adding the list of words/lemmas used for every song     



#cleaning and lemmatizing Lyrics_test:
Pro_Lyrics_test = pd.DataFrame(Lyrics_test).copy()
Pro_Lyrics_test['lyrics'] = Pro_Lyrics_test['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))#All the rows of the text in the data frame is checked for string punctuations, and these are filtered
Pro_Lyrics_test['lyrics'] = Pro_Lyrics_test['lyrics'].str.replace('[^\w\s]','')#REmoves dots using regular expressions
Pro_Lyrics_test['lyrics'] = Pro_Lyrics_test['lyrics'].apply(lambda x:' '.join(x.lower() for x in x.split())) #Converting text to lower case
Pro_Lyrics_test['lyrics'] = Pro_Lyrics_test['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))#Digits are removed from the text
Pro_Lyrics_test['lyrics'] = Pro_Lyrics_test['lyrics'].apply(lambda x: ' '.join(x for x in x.split() if not x in stop))#Stop words are removed at this stage
Pro_Lyrics_test['lyrics'] = Pro_Lyrics_test['lyrics'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))#Words are filtered now, and different form of the same word is removed using lemmatization
Pro_Lyrics_test_list = []
for i in Pro_Lyrics_test['lyrics']:
     li = list(i.split(" "))
     Pro_Lyrics_test_list.append(li)	
Pro_Lyrics_test['words'] = Pro_Lyrics_test_list #Adding the list of words/lemmas used for every song
################################################################################
#######  END OF PREPROCESSING  #################################################
################################################################################



######################  3  #####################################################
############# Training Phase ###################################################
################################################################################

##########################  Training a couple models  ###################
#Training a model with the vocabulary from Pro_Lyrics_list
RM_model = Word2Vec(Pro_Lyrics_list, min_count=2, size=150, workers=15, window=15)
print()
#Saving the model
RM_model.save("word2vec.RM_model")
RM_model.save("RM_model.bin")
print()

#Training a model with the imported vocabulary from abc.sents()
abc_model = gensim.models.Word2Vec(abc.sents(), min_count=2, size=150, workers=15, window=15)
print()
#Saving the model
abc_model.save("word2vec.abc_model")
abc_model.save("abc_model.bin")
print()


####################### Storing vectors gen by models #################
# Store the vectors for train data in following file
### Finish <----------------------------------------------------------------------------------------Incomplete
#word2vec_filename = OUTPUT_FOLDER + 'train_review_word2vec.csv'
#RM_vectors_filename = r'C:\Users\hprob\Desktop\ErdosProjectMay2020\Sample_project\RM_vectors.csv'
#with open(RM_vectors_filename, 'w+') as word2vec_file:
#    for index, row in Lyrics_train.iterrows():
#        model_vector = (np.mean([RM_model[token] for token in row['lyrics']], axis=0)).tolist()
#        if index == 0:
#            header = ",".join(str(ele) for ele in range(1000))
#            word2vec_file.write(header)
#            word2vec_file.write("\n")
#        # Check if the line exists else it is vector of zeros
#        if type(model_vector) is list:  
#            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
#        else:
#            line1 = ",".join([str(0) for i in range(1000)])
#        word2vec_file.write(line1)
#        word2vec_file.write('\n')
### Finish <----------------------------------------------------------------------------------------Incomplete
################################################################################
#######  END OF Training phase  #################################################
################################################################################






############################### 4 ##############################################
################## Data Analysis phase #########################################
################################################################################

##### Similarity testing and playing around ################
#word: flesh
abc_voc = list(abc_model.wv.vocab)
print("Most similar to flesh according to abc_model: ")
abc_data = abc_model.wv.most_similar('flesh')
print(abc_data)

RM_voc = list(RM_model.wv.vocab)
print("Most similar to flesh according to RM_model: ")
RM_data = RM_model.wv.most_similar('flesh')
print(RM_data)

### word: worm
abc_voc = list(abc_model.wv.vocab)
print("Most similar to worm according to abc_model: ")
abc_data = abc_model.wv.most_similar('worm')
print(abc_data)

RM_voc = list(RM_model.wv.vocab)
print("Most similar to worm according to RM_model: ")
RM_data = RM_model.wv.most_similar('worm')
print(RM_data)

### word: plastic
abc_voc = list(abc_model.wv.vocab)
print("Most similar to plastic according to abc_model: ")
abc_data = abc_model.wv.most_similar('plastic')
print(abc_data)

RM_voc = list(RM_model.wv.vocab)
print("Most similar to plastic according to RM_model: ")
RM_data = RM_model.wv.most_similar('plastic')
print(RM_data)

### word: hell
abc_voc = list(abc_model.wv.vocab)
print("Most similar to hella ccording to abc_model: ")
abc_data = abc_model.wv.most_similar('hell')
print(abc_data)

RM_voc = list(RM_model.wv.vocab)
print("Most similar to hell according to RM_model: ")
RM_data = RM_model.wv.most_similar('hell')
print(RM_data)

############ Similarity tests  ##############################
similar_words = abc_model.wv.most_similar('god')	
print(similar_words)
print()

similar_words = RM_model.wv.most_similar('god')	
print(similar_words)
print()
#############
dissimlar_words = abc_model.wv.doesnt_match('god good hell'.split())
print(dissimlar_words)
print()
dissimlar_words = RM_model.wv.doesnt_match('god good hell'.split())
print(dissimlar_words)
print()

#################
similarity_two_words = abc_model.wv.similarity('human','rat')
print("Please provide the similarity between these two words:")
print(similarity_two_words)
print()

similarity_two_words = RM_model.wv.similarity('human', 'rat')
print("Please provide the similarity between these two words:")
print(similarity_two_words)
print()

##########
similar = abc_model.wv.similar_by_word('kind')
print(similar)
print()
similar = RM_model.wv.similar_by_word('kind')
print(similar)
print()


###### Loading a model ################################
#model = Word2Vec.load('model.bin')
#vocab = list(model.wv.vocab)
#print()
#######################################################



############################################################
#########################  PCA #############################

######## passing vectors into df ######################
RM_model.wv.vocab
RM_model.wv.vectors.shape
RM_model.corpus_total_words

DF_RM_vectors = pd.DataFrame(data = RM_model.wv.vectors[0:,0:],
                index=[i for i in range(RM_model.wv.vectors.shape[0])],
                columns=['f'+str(i) for i in range(RM_model.wv.vectors.shape[1])])
DF_RM_vectors.tail()
 

abc_model.wv.vocab
abc_model.wv.vectors.shape
abc_model.corpus_total_words

DF_abc_vectors = pd.DataFrame(data = abc_model.wv.vectors[0:,0:],
                index=[i for i in range(abc_model.wv.vectors.shape[0])],
                columns=['f'+str(i) for i in range(abc_model.wv.vectors.shape[1])])
DF_abc_vectors.tail()
############################################################

########## Re-scaling data ######################################
### RM
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(RM_model.wv.vectors)
# Apply transform to both the training set and the test set.
scaled_RM_vectors = scaler.transform(RM_model.wv.vectors)
scaled_RM_vectors  = scaler.transform(RM_model.wv.vectors)

RM_pca = PCA(n_components=2)
RM_pca.fit(RM_model.wv.vectors)
RM_pca.n_components_

RM_pca.explained_variance_
scaled_RM_vectors = RM_pca.transform(RM_model.wv.vectors)

### abc
abc_scaler = StandardScaler()
# Fit on training set only.
abc_scaler.fit(abc_model.wv.vectors)
# Apply transform to both the training set and the test set.
scaled_abc_vectors = abc_scaler.transform(abc_model.wv.vectors)
scaled_RM_vectors  = abc_scaler.transform(abc_model.wv.vectors)

abc_pca = PCA(n_components=2)
abc_pca.fit(abc_model.wv.vectors)

abc_pca.explained_variance_
scaled_abc_vectors = abc_pca.transform(abc_model.wv.vectors)

#### plotting RM vocabulary  ###############
plt.figure(figsize=(20,20))
plt.scatter(scaled_RM_vectors[:, 0], scaled_RM_vectors[:, 1])
plt.xlabel("$PCA_1$", fontsize=14)
plt.ylabel("$PCA_2$", fontsize=14)
plt.axis('equal')
plt.show()
################## clustering ############
### specifying some colors
number = 100 # of colors/clusters
cmap = plt.get_cmap('gnuplot')
colors = [cmap(i) for i in np.linspace(0, 1, number)]

kmeans = KMeans(number)
kmeans.fit(scaled_RM_vectors)
RM_clusters = kmeans.predict(scaled_RM_vectors)


plt.figure(figsize=(20,20))
for i, color in enumerate(colors, start=1):
    plt.scatter(scaled_RM_vectors[RM_clusters==i,0], 
      scaled_RM_vectors[RM_clusters==i,1], label="$k$ Cluster " + str(i),color = color)
plt.legend(fontsize=14)
plt.show()


##### plotting abc vocabulary  ###############
plt.figure(figsize=(20,20))
plt.scatter(scaled_abc_vectors[:, 0], scaled_abc_vectors[:, 1])
plt.xlabel("$PCA_1$", fontsize=14)
plt.ylabel("$PCA_2$", fontsize=14)
plt.axis('equal')
plt.show()
################## clustering ############
### specifying some colors
number = 50 # of colors/clusters
cmap = plt.get_cmap('gnuplot')
colors = [cmap(i) for i in np.linspace(0, 1, number)]

kmeans = KMeans(number)
kmeans.fit(scaled_abc_vectors)
abc_clusters = kmeans.predict(scaled_abc_vectors)


plt.figure(figsize=(20,20))
for i, color in enumerate(colors, start=1):
    plt.scatter(scaled_abc_vectors[abc_clusters==i,0], 
      scaled_abc_vectors[abc_clusters==i,1], label="$k$ Cluster " + str(i),color = color)
plt.legend(fontsize=14)
plt.show()



##################################################################
###### Representing songs as vectors ############################
#dropping words NOT in vocabulary

count = 0
for index, row in Pro_Lyrics_train.iterrows():
    count = 0
    for w in row['words']:
        if w in RM_model.wv.vocab:
            print(row['words'][count], "row: ", count)            
            count = count+1           

Xx = np.zeros(0)
if type([1,2,3,]) is list:
        print("yes")

###present = np.empty(len(Pro_Lyrics_train), dtype=list)
        
####### checks for all words that were (not) vectorized ###
## stores 0 or 1 in "in vocabulary" column
turtle = []
present = [turtle for i in range(len(Pro_Lyrics_train))]
Pro_Lyrics_train['in_vocabulary'] = present
Pro_Lyrics_train.tail(5)


Zv = [np.zeros(150) for i in range(len(Pro_Lyrics_train))]
Pro_Lyrics_train['s_vector'] = Zv

Pro_Lyrics_train.sample(3)

### Looking for words in RM_model.vocab else, to then delete words
count = 0
for index, row in Pro_Lyrics_train.iterrows():
    count +=1
    for w in row['words']:
        if w in RM_model.wv.vocab:
            #print(RM_model.wv.word_vec(w))
            #print('smt')
            #print(row['in_vocabulary'])
            row['in_vocabulary'] = row['in_vocabulary'] + [1] #means w is part of RM_vocab
        else:
            row['in_vocabulary'] = row['in_vocabulary'] + [0] #means w is not part of RM_vocab
    if(count % 10000 == 0):
        print(count)
Pro_Lyrics_train.tail(3)

print(len(Pro_Lyrics_train['in_vocabulary']))
##################################




############### representing a song by the average of its words  ##################################
# Computing the average of vectorized words in "in vocabulary"
past_iter = np.zeros(150)
count = 0
num = 0
dummy = 0
Pro_Lyrics_train_song_mean = []
for index, row in Pro_Lyrics_train.iterrows():
    num = 0
    temp = np.zeros(150)
    #for i in row['in_vocabulary']:
    #    if(i == 1):
    #        #print(row['words'][i])#=w
    #        temp = temp + RM_model.wv.word_vec(row['words'][i])
    #        num += 1     
    dummy = 0    
    for w in row['words']:
        if(row['in_vocabulary'][dummy] == 1):
            #print("w: ", RM_model.wv.word_vec(w))
            #print(row['in_vocabulary'][dummy])
            #print(w)#row['words'][i])#=w
            #print("dummy: ", dummy)
            temp = temp + RM_model.wv.word_vec(w)
            num += 1
            #print("temp: ",temp)
            #print("num: ", num)
        #else:
            #print("problematic word!!")
            #print(row['in_vocabulary'][dummy])
            #print(w)#row['words'][i])#=w
            #print(dummy)    
        dummy += 1
    if(num == 0):    
        print("show me the money!")
        row['s_vector'] = np.zeros(150)        
    else:
        num = num*1.0
        #print("I know kung fu!")
        #print("s_vec: ", row['s_vector'])
        #if(count % 1000 == 0):
        #     print("s_vector: ", row['s_vector'])
        #     if ((past_iter==row['s_vector']).all()):
        #         print("No changes", )
        #     past_iter = row['s_vector']
        row['s_vector'] = temp/num
        #print("s_vec: ", row['s_vector'])
    count += 1
    if(count % 10000 == 0):
        print(count)
        print(num)
         
Pro_Lyrics_train.sample(30)
#######################################


print(Pro_Lyrics_train['words'][21564][0])
del Pro_Lyrics_train['words'][21564][0]
##################################################################



######### DecisionTree Classifier  ###############
Pro_Lyrics_train_clf = DecisionTreeClassifier(random_state = 440)
# Fit the model
Pro_Lyrics_train_clf.fit(list(Pro_Lyrics_train.s_vector), type_train.type)
###########

# Plot the fitted tree
plt.figure(figsize = (20,20))
fig = Pro_Lyrics_train_clf.fit(list(Pro_Lyrics_train.s_vector), type_train.type)
tree.plot_tree(fig,filled = True)
plt.show()
##################        

###present = np.empty(len(Pro_Lyrics_train), dtype=list)

###Saving the DT model:
#import pickle

#pkl_filename = "DT_prolyrics_model.pkl"
#with open(pkl_filename, 'wb') as file:
    #pickle.dump(model, file)

###

####### checks for all words that were (not) vectorized ###
## stores 0 or 1 in "in vocabulary" column
turtle = []
present = [turtle for i in range(len(Pro_Lyrics_test))]
Pro_Lyrics_test['in_vocabulary'] = present
Pro_Lyrics_test.tail(5)


Zv = [np.zeros(150) for i in range(len(Pro_Lyrics_test))]
Pro_Lyrics_test['s_vector'] = Zv

Pro_Lyrics_test.sample(3)

### Looking for words in RM_model.vocab else, to then delete words
count = 0
for index, row in Pro_Lyrics_test.iterrows():
    count +=1
    for w in row['words']:
        if w in RM_model.wv.vocab:
            #print(RM_model.wv.word_vec(w))
            #print('smt')
            #print(row['in_vocabulary'])
            row['in_vocabulary'] = row['in_vocabulary'] + [1] #means w is part of RM_vocab
        else:
            row['in_vocabulary'] = row['in_vocabulary'] + [0] #means w is not part of RM_vocab
    if(count % 10000 == 0):
        print(count)
Pro_Lyrics_test.sample(5)

print(len(Pro_Lyrics_test['in_vocabulary']))
################


############### representing a song by the average of its words  ##################################
# Computing the average of vectorized words in "in vocabulary"
past_iter = np.zeros(150)
count = 0
num = 0
dummy = 0
Pro_Lyrics_test_song_mean = []
for index, row in Pro_Lyrics_test.iterrows():
    num = 0
    temp = np.zeros(150)
    dummy = 0    
    for w in row['words']:
        if(row['in_vocabulary'][dummy] == 1):
            temp = temp + RM_model.wv.word_vec(w)
            num += 1
        dummy += 1
    if(num == 0):    
        print("show me the money!")
        row['s_vector'] = np.zeros(150)        
    else:
        num = num*1.0
        row['s_vector'] = temp/num
    count += 1
    if(count % 1000 == 0):
        print(count)
        print(num)
        
Pro_Lyrics_test.sample(10)        
#######################################################       

######################### Making predictions with Tree ########
test_predictions_word2vec = Pro_Lyrics_train_clf.predict(list(Pro_Lyrics_test['s_vector']))

print(classification_report(type_test['type'], test_predictions_word2vec))
###############################################################


In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 64

# Edit this!!!!:
maxlen = 184
train_songs_padded = tf.keras.preprocessing.sequence.pad_sequences(songlist, maxlen=maxlen, padding='post').astype('float32')

train_songs_padded = tf.expand_dims(train_songs_padded, axis = -1)
train_songs_padded.shape

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_songs_padded).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# Creating a custom activation, a rescaling/shift of tanh so the range is (0,1).
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

def positive_tanh(x):
  return 0.5*(tf.nn.tanh(x) + 1)

get_custom_objects().update({'positive_tanh': Activation(positive_tanh)})

In [ ]:
# Generator model:

def make_generator_model(r):

    init = RandomNormal(stddev=0.02)

    model = tf.keras.Sequential()
    
    model.add(layers.Dense(23*4*256, use_bias=False, kernel_initializer=init, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((23, 4, 256)))
    assert model.output_shape == (None, 23, 4, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(1, 1), padding='same', use_bias=False, kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.L2(l2=r))) # was 5,5
    assert model.output_shape == (None, 23, 4, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.L2(l2=r)))
    assert model.output_shape == (None, 46, 8, 64) #1
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.L2(l2=r)))
    assert model.output_shape == (None, 92, 16, 32) #1
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # activation was here, and was tanh. We changed to positive_tanh.
    model.add(layers.Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', use_bias=False, kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.L2(l2=r)))
    model.add(Activation(positive_tanh, name='positive_tanh'))
    assert model.output_shape == (None, 184, 32, 1)


    return model

In [ ]:
# This is the final activator, turning the array of floats into 0s and 1s, so that 
# it can actually be read/written as a MIDI file. We have a choice of 4 different rectifiers,
# each deciding whether m[i,j] = 0 or 1 based on the matrix's average, nth percentile,
# an average of a travelling window, or a fixed value.

def avg_rectifier(T):
  m = np.array(T)

  avg = np.mean(m)

  new_m = np.zeros(np.shape(m))

  for i in range(m.shape[0]):
    for j in range(m.shape[1]):
      if m[i,j] > avg:
        new_m[i,j] = 1
      else:
        new_m[i,j] = 0

  new_m = tf.expand_dims(new_m, axis = -1)
  new_m = tf.expand_dims(new_m, axis = 0)

  return new_m


def percentile_rectifier(T, perc):
  m = np.array(T)

  # This gets the perc^th percentile:
  percentile = np.percentile(m,perc)

  new_m = np.zeros(np.shape(m))

  for i in range(m.shape[0]):
    for j in range(m.shape[1]):
      if m[i,j] > percentile:
        new_m[i,j] = 1
      else:
        new_m[i,j] = 0

  new_m = tf.expand_dims(new_m, axis = -1)
  new_m = tf.expand_dims(new_m, axis = 0)

  return new_m


def window_rectifier(T, pad):
  m = np.array(T)

  new_m = np.zeros(np.shape(m))

  for i in range(pad,m.shape[0]-pad):
    for j in range(pad,m.shape[1]-pad):
      # Here we take the average of a (2*pad+1)x(2*pad+1) window around m[i,j]
      L = m[i-pad:i+pad+1,j-pad:j+pad+1]
      L[pad,pad] = 0
      window_avg = np.sum(L)/((2*pad+1)^2 - 1)

      if m[i,j] > window_avg:
        new_m[i,j] = 1
      else:
        new_m[i,j] = 0
  
  # Here we're losing the information on the edges:
  new_m[:pad,:pad] = 0
  new_m[m.shape[0]-pad:,m.shape[1]-pad:] = 0

  new_m = tf.expand_dims(new_m, axis = -1)
  new_m = tf.expand_dims(new_m, axis = 0)

  return new_m


def absolute_rectifier(T, value):
  m = np.array(T)

  new_m = np.zeros(np.shape(m))
  # value is what decides whether m[i,j] = 0 or 1

  for i in range(m.shape[0]):
    for j in range(m.shape[1]):
      if m[i,j] > value:
        new_m[i,j] = 1
      else:
        new_m[i,j] = 0

  new_m = tf.expand_dims(new_m, axis = -1)
  new_m = tf.expand_dims(new_m, axis = 0)

  return new_m


In [ ]:
# implementation of wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

In [ ]:
# Losses:
# Note -1 = real, 1 = fake.

def discriminator_loss_total(real_output, fake_output):
    real_loss = wasserstein_loss(-1*tf.ones_like(real_output), real_output)
    fake_loss = wasserstein_loss(tf.ones_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def discriminator_loss_real(real_output):
    real_loss = wasserstein_loss(-1*tf.ones_like(real_output), real_output)
    return real_loss

def discriminator_loss_fake(fake_output):
    fake_loss = wasserstein_loss(tf.ones_like(fake_output), fake_output)
    return fake_loss

def generator_loss(fake_output):
    return wasserstein_loss(-1*tf.ones_like(fake_output), fake_output)

In [ ]:
# DCGANs usually use 100,000 - 500,000 iterations (epochs) for the generator...Colab pro?
EPOCHS = 25

noise_dim = 100
# make this a square (1,4,16,... etc.) for the subplotting code:
num_examples_to_generate = 1

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
def train(dataset, epochs, val, tag, w1, w2, w3):
    n_critic = 5
    c_r_hist, c_f_hist, g_hist = list(), list(), list()
    # Should the noise go here??? And pass the same noise to the d_train_step and g_train_step???
    count = 0

    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            c_r_tmp, c_f_tmp, g_tmp = list(), list(), list()

            for _ in range(n_critic):
                c_r_loss, c_f_loss = d_train_step(image_batch)
                c_r_tmp.append(c1_r_loss)
                c_f_tmp.append(c1_f_loss)

            c_r_hist.append(mean(c_r_tmp))
            c_f_hist.append(mean(c_f_tmp))

            g_loss = g_train_step(w1, w2, w3)
            g_hist.append(g_loss)

            print('>%d, c1r=%.3f, c1f=%.3f, c2r=%.3f, c2f=%.3f, c3r=%.3f, c3f=%.3f, g=%.3f' % (count+1, c1_r_hist[-1], c1_f_hist[-1], c2_r_hist[-1], c2_f_hist[-1], c3_r_hist[-1], c3_f_hist[-1], g_loss))

            count += 1

        # Save a song every 10 epochs:
        #display.clear_output(wait=True)
        if (epoch + 1) % 10 == 0:
          generate_and_save(generator, epoch + 1, seed, tag, val)

        # Save the model every 5 epochs
        if (epoch + 1) % 5 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Plot the (final) loss history
    plot_history(c1_r_hist, c1_f_hist, c2_r_hist, c2_f_hist, c3_r_hist, c3_f_hist, g_hist)
    plot_crit_history(c1_r_hist, c1_f_hist, c2_r_hist, c2_f_hist, c3_r_hist, c3_f_hist)
    plot_crit_fake_history(c1_f_hist, c2_f_hist, c3_f_hist)
    plot_crit_fake_avg_history(c1_f_hist, c2_f_hist, c3_f_hist)

    # Generate after the final epoch
    #display.clear_output(wait=True)
    generate_and_save(generator, epochs, seed, tag, val)

    return c_r_hist, c_f_hist, g_hist

In [ ]:
def generate_and_save(model, epoch, test_input, tag, val):
    # Notice 'training' is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(int(np.sqrt(num_examples_to_generate)), int(np.sqrt(num_examples_to_generate))))

    for i in range(predictions.shape[0]):
        plt.subplot(int(np.sqrt(num_examples_to_generate)), int(np.sqrt(num_examples_to_generate)), i+1)
        if tag == "absolute":
          plt.imshow(absolute_rectifier(predictions[i,:,:,0], val)[0,:,:,0], cmap='gray')
          plt.axis('off')
          noteStateMatrixToMidi(absolute_rectifier(predictions[i,:,:,0], val)[0,:,:,0], "song " + str(i) + " of epoch " + str(epoch))
        elif tag == "percentile":
          plt.imshow(percentile_rectifier(predictions[i,:,:,0], val)[0,:,:,0], cmap='gray')
          plt.axis('off')
          noteStateMatrixToMidi(percentile_rectifier(predictions[i,:,:,0], val)[0,:,:,0], "song " + str(i) + " of epoch " + str(epoch))
        elif tag == "window":
          plt.imshow(window_rectifier(predictions[i,:,:,0], val)[0,:,:,0], cmap='gray')
          plt.axis('off')
          noteStateMatrixToMidi(window_rectifier(predictions[i,:,:,0], val)[0,:,:,0], "song " + str(i) + " of epoch " + str(epoch))
        elif tag == "average":
          plt.imshow(avg_rectifier(predictions[i,:,:,0])[0,:,:,0], cmap='gray')
          plt.axis('off')
          noteStateMatrixToMidi(avg_rectifier(predictions[i,:,:,0])[0,:,:,0], "song " + str(i) + " of epoch " + str(epoch))

    plt.savefig('/content/drive/MyDrive/Music GAN Images/Images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
# the 3 fields for the generator are: 
# (L2 regularization parameter for the kernel, T/F value of whether to use kernel constraint, MaxNorm value for the kernel
# constraint [if included using the T/F value]).

# Here we make the generator and apply it to sample noise:
generator = make_generator_model(0.0001)
noise = tf.random.normal([1, 100])
S = generator(noise, training=False)

plt.imshow(S[0, :, :, 0], cmap='gray')

In [ ]:
# Now apply the rectifier to make all entries 0 or 1:
newS = percentile_rectifier(S[0,:,:,0], 80)
plt.imshow(newS[0,:,:,0], cmap='gray')

# Save the test song:
noteStateMatrixToMidi(np.array(newS[0,:,:,0]), "test song")

In [ ]:
# Just checking what a real song looks like:
plt.imshow(songlist[5][:184,:], cmap='gray')

In [ ]:
# the 4 fields for the discriminators are: 
# (L2 regularization parameter for the kernel, T/F value of whether to use a kernel constraint, MaxNorm value for the kernel
# constraint [if included using the T/F value], and dropout rate).

discriminator1 = make_discriminator_model1(0.0001, 0.3)   # Dropout was 0.3 earlier
discriminator2 = make_discriminator_model2(0.0001, 0.3)
discriminator3 = make_discriminator_model3(0.0001, 0.3)
decision1 = discriminator1(newS)
decision2 = discriminator2(newS)
decision3 = discriminator3(newS)
print(np.array(decision1)[0][0], np.array(decision2)[0][0], np.array(decision3)[0][0])
print()
# positive for real, negative for fake.

# weighted sum of 3 decisions:
weight1 = 1
weight2 = 1.5
weight3 = 0.7
print((weight1*np.array(decision1)[0][0] + weight2*np.array(decision2)[0][0] + weight3*np.array(decision3)[0][0])/(weight1+weight2+weight3))

In [ ]:
# Try a learning rate schedule (exponential decreasing cyclic)

#initial_learning_rate = 0.1
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, 
                                                          #decay_steps=100000, 
                                                          #decay_rate=0.96, 
                                                          #staircase=True)

lr_schedule = tfa.optimizers.ExponentialCyclicalLearningRate(initial_learning_rate=1e-3,
                                                             maximal_learning_rate=1e-1,
                                                             step_size=2000,
                                                             scale_mode="cycle",
                                                             gamma=0.90,
                                                             name="MyCyclicScheduler")

generator_optimizer = RMSprop(learning_rate=lr_schedule)
discriminator_optimizer1 = RMSprop(learning_rate=lr_schedule)
discriminator_optimizer2 = RMSprop(learning_rate=lr_schedule)
discriminator_optimizer3 = RMSprop(learning_rate=lr_schedule)

import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer1=discriminator_optimizer1,
                                 discriminator_optimizer2=discriminator_optimizer2,
                                 discriminator_optimizer3=discriminator_optimizer3,
                                 generator=generator,
                                 discriminator1=discriminator1,
                                 discriminator2=discriminator2,
                                 discriminator3=discriminator3)

In [ ]:
# Now for actual training:

# Last three numbers are weights for discriminator 1, 2, and 3. The generator's loss function is fed the weighted average of
# the decision from the 3 discriminators. Ex.: (w1, w2, w3) = (1, 0, 0) means the generator's loss function is only being
# fed the 1st discriminator's decision.
# The input before the weights is a tag which identifies which rectifier to use to make sure all entries are 0 or 1.
# Available rectifiers: absolute, percentile, window, and average.
# Before this is a value which, for the window/absolute/percentile rectifiers, specifies the window padding/
# fixed value/percentile which those rectifiers will use.

# QUESTION: Should training be True for all parties in each of d_train and g_train??? Or only the neural nets that /are/ being trained there???
# What about the "trainable" attribute?
c_r_hist,c_f_hist,g_hist = train(train_dataset, EPOCHS, 90, "percentile", 1, 1, 0.8)

# Best results I have found have been with percentile rectifier, 75-95th percentile and
# absolute rectifier with fixed value of around 0.15.

In [ ]:
plt.figure(figsize = (10,10))

plt.plot(c_r_hist, label="critic real loss")
plt.plot(c_f_hist, label="critic fake loss")
plt.plot(g_hist, label="generator fake loss")

plt.legend()

plt.show()